In [ ]:
import os

import numpy as np
import nengo
import nengo.utils.numpy as npext
from nengo_gui.ipython import IPythonViz

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
plt.rc('figure', figsize=(12, 3))

import phd

In [ ]:
%%javascript
if($(IPython.toolbar.selector.concat(' > #kill-run-first')).length == 0){
  IPython.toolbar.add_buttons_group([
    {
      'label'   : 'kill and run-first',
      'icon'    : 'fa fa-angle-double-down',
      'callback': function(){
        IPython.notebook.kernel.restart();
        $(IPython.events).one('kernel_ready.Kernel', function(){
          var idx = IPython.notebook.get_selected_index();
          IPython.notebook.select(0);
          IPython.notebook.execute_cell();
          IPython.notebook.select(idx);
        });
      }
    }
  ], 'kill-run-first');
}

## Trajectory detection

Basic idea:

- We know the trajectory through time,
  and use this to make continuous predictions of the trajectory.
- When the actual trajectory matches our predictions, we advance time.
- We can interrogate how far along the trajectory we are.

In [ ]:
import nengo.utils.numpy as npext

def predict(x):
    # Simple trajectory that just goes through 4 discrete states
    if x <= 0.25:
        return np.array([0, 0, 1, 1]).astype(float)
    elif x <= 0.5:
        return np.array([0, 1, 0, 1]).astype(float)
    elif x <= 0.75:
        return np.array([1, 0, 1, 0]).astype(float)
    else:
        return np.array([1, 1, 0, 0]).astype(float)

def observe(t):
    # Same trajectory, but at various points in time
    if t <= 0.5:
        return np.array([0, 0, 1, 1]).astype(float)
    elif t <= 0.6:
        return np.array([0, 1, 0, 1]).astype(float)
    elif t <= 0.7:
        return np.array([1, 0, 1, 0]).astype(float)
    else:
        return np.array([1, 1, 0, 0]).astype(float)

def similarity(v1, v2):
    # v1 and v2 are vectors
    eps = np.nextafter(0, 1)  # smallest float above zero
    dot = np.dot(v1, v2)
    dot /= max(npext.norm(v1), eps)
    dot /= max(npext.norm(v2), eps)
    return dot

# Go from time 0 to 2
t = np.linspace(0, 1, num=200)
# State
x = 0.
c = 0.02
x_hist = []
for tt in t:
    # Make a prediction, get an observation
    pred = np.hstack([[x], predict(x)])
    obs = np.hstack([[x], observe(tt)])

    # Increment x
    x += (similarity(pred, obs) > 0.9) * c
    x = np.clip(x, 0, 1)
    x_hist.append(x)

plt.plot(t, x_hist)

## Trajectory detection in Nengo

In [ ]:
trajs = []
gests = ('pat', 'das')
for ges in gests:
    path = phd.ges_path('ges-de-cvc', '%s.ges' % ges.lower())
    trajs.append(phd.vtl.parse_ges(path).trajectory(dt=0.001))

model = phd.sermo.Recognition()
model.trial.trajectory = trajs[0]
freqs = [('pat', 2.5), ('das', 4.7)]
for ges, freq in freqs:
    path = phd.ges_path('ges-de-cvc', '%s.ges' % ges.lower())
    traj = phd.vtl.parse_ges(path).trajectory(dt=model.trial.dt)
    model.add_syllable(freq=freq, trajectory=traj)

In [ ]:
net = model.build()
with net:
    p_traj = nengo.Probe(net.trajectory.output, synapse=0.01)
    p_dmps = [nengo.Probe(dmp.state, synapse=0.01) for dmp in net.syllables]

In [ ]:
sim = nengo.Simulator(net)
sim.run(1.0)

t = sim.trange()
plt.figure()
plt.plot(t, sim.data[p_traj])
for pr, label in zip(p_dmps, gests):
    plt.figure()
    plt.plot(t, sim.data[pr])
    plt.title(label)

In [ ]:
def sp_to_assocmem(sp=None):
    # Random SP
    sp = nengo.spa.SemanticPointer(16).v if sp is None else sp
    def to_assocmem(x):
        if x[0] > 0.8:
            return sp
        else:
            return np.zeros_like(sp)
    to_assocmem.sp = sp
    return to_assocmem

In [ ]:
from phd.networks.dmp import traj2func

ffuncs = [traj2func(traj, dt=0.001)[0] for traj in trajs]
traj_dims = trajs[0].shape[1]

with nengo.Network() as net:
    # Emulate the associative memory input, for now.
    assoc_mem_in = nengo.networks.EnsembleArray(25, 16)

    sps, ensembles, probes = [], [], []
    for f in ffuncs:
        # recover `x` ramp from forced function
        forced_inv = nengo.Ensemble(200*(f.ix.size+1), dimensions=f.ix.size+1,
                                    n_eval_points=10000, radius=1.4)
        ensembles.append(forced_inv)
        # first dimension is a recurrent connection,
        # advancing x or not depending on the input observation
        nengo.Connection(forced_inv, forced_inv[0],
                         function=ff_inv(f), synapse=0.05)
        # last three dimensions are getting input observations
        nengo.Connection(dmp_out.output[f.ix], forced_inv[1:])

        # Reset with the kick
        nengo.Connection(reset, forced_inv.neurons, synapse=.01,
                         transform=-np.ones((forced_inv.n_neurons, 1)))

        # Connect to associative memory
        ffi_toassocmem = sp_to_assocmem()
        sps.append(ffi_toassocmem.sp)
        nengo.Connection(forced_inv, assoc_mem_in.input, function=ffi_toassocmem)

        # Probe
        probes.append(nengo.Probe(forced_inv, synapse=0.01))

    # Introduce some minor (friendly!) competition
    comp_scale = 0.05
    for ens1 in ensembles:
        for ens2 in ensembles:
            if ens1 is not ens2:
                nengo.Connection(ens1[0], ens2[0], transform=-comp_scale)

    p_ff = nengo.Probe(dmp.osc, synapse=0.01)
    p_forced = nengo.Probe(dmp_out.output, synapse=0.01)
    p_assocmem = nengo.Probe(assoc_mem_in.output, synapse=0.01)

In [ ]:
sim = nengo.Simulator(net)
sim.run(1.0)

t = sim.trange()
plt.figure()
plt.plot(t, sim.data[p_ff])
plt.figure()
plt.plot(t, sim.data[p_forced])
for pr, label in zip(probes, gests):
    plt.figure()
    plt.plot(t, sim.data[pr])
    plt.title(label)
plt.figure()
plt.plot(t, nengo.spa.similarity(sim.data[p_assocmem], sps, True))
plt.legend(gests, loc='best')